**Tras no conseguir mejorar hasta el momento, se ca a cambiar de estrategia. Se van a añadir más variables para la predicción del primer modelo en el que se aplica Ridge en primera instancia**

In [18]:
# Setup notebook
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.graph_objects as go
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import warnings
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import make_scorer, r2_score, mean_squared_error
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from joblib import Parallel, delayed
import category_encoders as ce
from matplotlib.offsetbox import AnchoredText
import math
from sklearn.preprocessing import LabelEncoder

In [19]:
store_sales_train = pd.read_csv('train.csv')
store_sales_test = pd.read_csv('test.csv')
oil = pd.read_csv('oil.csv', parse_dates=['date'])
holidays_events = pd.read_csv('holidays_events.csv', dtype={'type': 'category','locale': 'category',
                            'locale_name': 'category','description': 'category','transferred': 'bool',}, parse_dates=['date'], infer_datetime_format=True)
transactions = pd.read_csv('transactions.csv', parse_dates=['date'])
stores = pd.read_csv('stores.csv')

In [20]:
oil['date'] = pd.to_datetime(oil['date'])
oil['date'] = oil.date.dt.to_period('D')
oil.set_index('date', inplace=True)
oil = oil.rename(columns={'dcoilwtico':'precio'})
#Se rellenan los nulos correspondientes al fin de semana con el valor del viernes
oil['precio'].ffill(inplace=True)

In [21]:
def make_lags_precio(ts, lags):
    return pd.concat(
        {
            f'y_lag_precio_{i}': ts.shift(i)
            for i in range(1, lags + 1)
        },
        axis=1)

In [22]:
lags_oil = make_lags_precio(oil['precio'],4)
oil = oil.join(lags_oil)
oil['precio_medio_semanal'] = oil['precio'].rolling(7).mean()
oil = oil.loc['2016-06-01':,:]

In [23]:
store_sales_train['date'] = pd.to_datetime(store_sales_train['date'])
store_sales_train['date'] = store_sales_train.date.dt.to_period('D')
store_sales_test['date'] = pd.to_datetime(store_sales_test['date'])
store_sales_test['date'] = store_sales_test.date.dt.to_period('D')

store_sales_train_17 = store_sales_train.loc[store_sales_train['date'] >= '2016-06-01'].set_index(['date'])
store_sales_test = store_sales_test.set_index('date')

In [24]:
df_complete = pd.concat([store_sales_train_17,store_sales_test])

Se añaden más temporalidades; anual, mensual y semanal. Se añaden los datos del precio del petróleo, las vacaciones tanto nacionales como locales y regionales, pues se va a elaborar un modelo por tienda, de forma que las fiestas regionales y locales se pueden incluir. Además se van a incluir descripciones de los eventos y se crearán dummiesde las mismas, para diferenciar así eventos importantes como el Black Friday que pueden influir en las ventas

In [25]:
fourierA = CalendarFourier(freq='A', order=5)
fourierM = CalendarFourier(freq='M', order=2)
fourierW = CalendarFourier(freq='W', order=4)

dp = DeterministicProcess(
    index=df_complete.index,
    constant=False,
    order=1,
    seasonal=False,
    additional_terms=[fourierA, fourierM, fourierW],
    drop=True
)
df_dp = dp.in_sample()
df_complete = pd.concat([df_complete, df_dp], axis=1)

df_complete = df_complete.join(oil, on='date')
df_complete['precio'].ffill(inplace=True)
df_complete['y_lag_precio_1'].ffill(inplace=True)
df_complete['y_lag_precio_2'].ffill(inplace=True)
df_complete['y_lag_precio_3'].ffill(inplace=True)
df_complete['y_lag_precio_4'].ffill(inplace=True)
df_complete['precio_medio_semanal'].fillna(df_complete['precio'].rolling(7).mean(), inplace=True)
df_complete['precio_medio_semanal'].ffill(inplace=True)

In [94]:
df_complete = df_complete.reset_index().merge(stores, on=['store_nbr'], how='left')

In [99]:
#holidays_events['date'] = pd.to_datetime(holidays_events['date'])
#holidays_events['date'] = holidays_events.date.dt.to_period('D')

holidays_17 = holidays_events.query("transferred==False").loc[holidays_events['date'].isin(pd.period_range('2016-06-01','2017-08-31'))]
holidays_17 = holidays_17.drop(index=holidays_17[holidays_17[['date', 'locale_name']].duplicated()].index.values)
#Adding descriptions to events that are not holidays
holidays_17.rename(columns={'type':'event_type'}, inplace=True)
holidays_17['event_type'] = holidays_17['event_type'].astype('object')
holidays_17.loc[holidays_17.event_type=='Event', 'event_type'] = holidays_17.description.apply(lambda x: x[0:7])

nat_holidays = holidays_17.query("locale=='National'")
reg_holidays = holidays_17.query("locale=='Regional'")
loc_holidays = holidays_17.query("locale=='Local'")

df_complete = df_complete.merge(nat_holidays[['event_type','date']], left_on=['date'], right_on=['date'], how='left')
df_complete = df_complete.merge(reg_holidays[['event_type','date', 'locale_name']], left_on=['date', 'state'], right_on=['date', 'locale_name'], how='left')
df_complete = df_complete.merge(loc_holidays[['event_type','date', 'locale_name']], left_on=['date', 'city'], right_on=['date', 'locale_name'], how='left')

df_complete.loc[~df_complete.event_type_x.isnull(), 'description'] = df_complete.event_type_x.apply(lambda x: x)
df_complete.loc[~df_complete.event_type_y.isnull(), 'description'] = df_complete.event_type_y.apply(lambda x: x)
df_complete.loc[~df_complete.event_type.isnull(), 'description'] = df_complete.event_type.apply(lambda x: x)
df_complete.loc[df_complete.description.isnull(), 'description'] = df_complete.description.apply(lambda x: 'norm')
df_complete = df_complete.drop(columns=['event_type_x', 'event_type_y', 'event_type'])

df_complete['isevent'] = df_complete.description.apply(lambda x: 'y' if x!='norm' else 'n')

df_complete['isclosed'] = df_complete.groupby(by=['date', 'store_nbr'])['sales'].transform(lambda x: 1 if x.sum()==0 else 0)    
df_complete.loc[(df_complete.date.dt.year==2017) & (df_complete.date.dt.month==8) & (df_complete.date.dt.day>=16) , 'isclosed'] = df_complete.isclosed.apply(lambda x: 0)    
df_complete.loc[df_complete.date.isin(['2017-01-01']), 'isevent'] = df_complete.isevent.apply(lambda x: 'n')

df_complete.drop(['locale_name_x', 'locale_name_y'], axis = 1, inplace=True)
df_complete.set_index('date', inplace=True)

Se va a incluir una columna con el número total de promociones en cada tienda por día

In [106]:
df_complete['tot_store_day_onprom'] = df_complete.groupby(by=['date', 'store_nbr']).onpromotion.transform(lambda x: x.sum())

In [ ]:
df_complete['year'] = df_complete.index.year.astype('int')
df_complete['quarter'] = df_complete.index.quarter.astype('int')
df_complete['month'] = df_complete.index.month.astype('int')
df_complete['day'] = df_complete.index.day.astype('int')
df_complete['dayofweek'] = df_complete.index.day_of_week.astype('int')
df_complete['weekofyear'] = df_complete.index.week.astype('int')
df_complete['isweekend'] = df_complete.dayofweek.apply(lambda x: 1 if x in (5,6) else 0)
df_complete['startschool'] = df_complete.month.apply(lambda x: 1 if x in (4,5,8,9) else 0)
df_complete['dayofyear'] = df_complete.index.dayofyear.astype('int')
df_complete['NewYear'] = df_complete.dayofyear.apply(lambda x: 1 if x==1 else 0)

#Ejecutado hasta aquí
df_complete = pd.get_dummies(df_complete, columns=['year'], drop_first=True)
df_complete = pd.get_dummies(df_complete, columns=['quarter'], drop_first=True)
df_complete = pd.get_dummies(df_complete, columns=['type'], drop_first=True)
df_complete = pd.get_dummies(df_complete, columns=['description'], drop_first=True)
df_complete = pd.get_dummies(df_complete, columns=['isevent'], drop_first=True)

df_complete.drop(['month','dayofyear'], axis=1, inplace=True)

In [120]:
df_complete

,id,store_nbr,family,sales,onpromotion,trend,"sin(1,freq=A-DEC)","cos(1,freq=A-DEC)","sin(2,freq=A-DEC)","cos(2,freq=A-DEC)",...,type_E,description_Black F,description_Bridge,description_Cyber M,description_Dia de,description_Holiday,description_Transfer,description_Work Day,description_norm,isevent_y
date,,,,,,,,,,,,,,,,,,,,,
2016-06-01,2216808,1,AUTOMOTIVE,3.0,0,1.0,0.507415,-0.861702,-0.874481,0.485060,...,0,0,0,0,0,0,0,0,1,0
2016-06-01,2216809,1,BABY CARE,0.0,0,2.0,0.507415,-0.861702,-0.874481,0.485060,...,0,0,0,0,0,0,0,0,1,0
2016-06-01,2216810,1,BEAUTY,4.0,0,3.0,0.507415,-0.861702,-0.874481,0.485060,...,0,0,0,0,0,0,0,0,1,0
2016-06-01,2216811,1,BEVERAGES,2199.0,37,4.0,0.507415,-0.861702,-0.874481,0.485060,...,0,0,0,0,0,0,0,0,1,0
2016-06-01,2216812,1,BOOKS,0.0,0,5.0,0.507415,-0.861702,-0.874481,0.485060,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-31,3029395,9,POULTRY,NaN,1,812588.0,-0.854322,-0.519744,0.888057,-0.459733,...,0,0,0,0,0,0,0,0,1,0
2017-08-31,3029396,9,PREPARED FOODS,NaN,0,812589.0,-0.854322,-0.519744,0.888057,-0.459733,...,0,0,0,0,0,0,0,0,1,0
2017-08-31,3029397,9,PRODUCE,NaN,1,812590.0,-0.854322,-0.519744,0.888057,-0.459733,...,0,0,0,0,0,0,0,0,1,0


Para el modelo Ridge no se tendrán en cuenta los lags del precio del petróleo ni los datos de las promociones (empeoraba resultado). Se seleccionan las columnas cols concretamente. Como se elabora un modelo por tienda, se utiliza el mismo dataset X para cada una de ellas independientemente de la familia de productos (la y se pone en función de la familia, de forma que se ajuste un modelo a cada una de ellas).

In [182]:
cols = df_complete.columns[df_complete.columns.str.match(r'year|quarter|description|isevent|isclosed|week|isweek|startschool|sin|cos|trend')]

X_Ridge = {}
X_train_Ridge = {}
X_test_Ridge = {}
y_Ridge = {}
y_train_Ridge = {}
y_test_Ridge = {}

for store in df_complete.store_nbr.unique(): 
    X_Ridge[store] = df_complete.loc[df_complete['store_nbr']==store,cols]
    X_Ridge[store] = X_Ridge[store].groupby(by='date').first()
    X_train_Ridge[store] = X_Ridge[store].loc[:'2017-08-15']
    X_test_Ridge[store] = X_Ridge[store].loc['2017-08-16':]
    y_Ridge[store] = df_complete.loc[df_complete['store_nbr']==store,['sales','family','store_nbr']].reset_index().set_index(['date','store_nbr','family']).unstack('family')
    y_train_Ridge[store] = np.log1p(y_Ridge[store].loc[:'2017-08-15'])
    y_test_Ridge[store] = y_Ridge[store].loc['2017-08-16':]

Se le da más valor para hacer el fit a los datos de 2017 que a los de 2016 (weights)

In [183]:
model_Ridge = {}
y_pred_store_Ridge = {}
y_fore_store_Ridge = {}
y_pred_Ridge = pd.DataFrame()
y_fore_Ridge = pd.DataFrame()
for store in df_complete.store_nbr.unique():
    weights = X_train_Ridge[store].year_2017.apply(lambda x: np.exp((-0.9)*1) if x == 0 else np.exp((-0.9)*0))
    model_Ridge[store] = Ridge(fit_intercept=False)
    model_Ridge[store].fit(X_train_Ridge[store], y_train_Ridge[store], sample_weight=weights)
    y_pred_store_Ridge[store] = pd.DataFrame(model_Ridge[store].predict(X_train_Ridge[store]), index= y_train_Ridge[store].index, columns=y_train_Ridge[store].columns)
    y_pred_Ridge = pd.concat([y_pred_Ridge, y_pred_store_Ridge[store]])
    y_fore_store_Ridge[store] = pd.DataFrame(model_Ridge[store].predict(X_test_Ridge[store]), index= y_test_Ridge[store].index, columns=y_test_Ridge[store].columns)
    y_fore_Ridge = pd.concat([y_fore_Ridge, y_fore_store_Ridge[store]])

In [184]:
y_pred_Ridge = y_pred_Ridge.stack('family').sort_index()
y_fore_Ridge = y_fore_Ridge.stack('family').sort_index()

In [185]:
y_pred_Ridge = np.exp(y_pred_Ridge)-1
y_fore_Ridge = np.exp(y_fore_Ridge)-1

In [190]:
y_fore_Ridge

sales
date       store_nbr family                                 
2017-08-16 1         AUTOMOTIVE                     3.869318
                     BABY CARE                      0.000000
                     BEAUTY                         3.638269
                     BEVERAGES                   2857.123028
                     BOOKS                          0.304505
...                                                      ...
2017-08-31 54        POULTRY                       54.016150
                     PREPARED FOODS                82.592075
                     PRODUCE                      549.287980
                     SCHOOL AND OFFICE SUPPLIES     0.000000
                     SEAFOOD                        1.578420

[28512 rows x 1 columns]

In [226]:
y_fore_Ridge[y_fore_Ridge<0]=0
store_sales_test = store_sales_test.reset_index().set_index(['date','store_nbr','family']).sort_index()
y_submit = y_fore_Ridge.join(store_sales_test.id)
y_submit.columns = ['sales','id']
y_submit = y_submit.set_index('id')
y_submit.sort_index(inplace=True)
y_submit

,sales
id,
3000888,3.869318
3000889,0.000000
3000890,3.638269
3000891,2857.123028
3000892,0.304505
...,...
3029395,398.841310
3029396,132.383365
3029397,1504.446545


In [188]:
y_submit.to_csv('submission14.csv')

**El RMSLE ha sido de 0,4435, claramente mejor a cualquiera de los que se tenían hasta ahora con la estrategia anterior**

# Se va a tratar de mejorar los resultados promediando con aquellos obtenidos mediante un RandomForest

Para el RF se van a implementar también las columnas de los lags y de las promociones, de forma que esta información que no ha tenido en cuenta en la primera predicción sí se pueda tener en cuenta aquí

In [211]:
dict_params = {"n_estimators": [200, 250, 300],
               "max_depth": [5, 10, 15, 20],
               "criterion": ["mse"],
               "max_features": [None],
               "bootstrap": [True],
               "oob_score": [True],
               "max_samples": [0.6]
              }

In [214]:
cols = df_complete.columns[df_complete.columns.str.match(r'year|quarter|description|onpromotion|tot_|isevent|isclosed|precio|y_lag|week|isweek|startschool|sin|cos|trend')]
#Each shop has the same X for every family, so a model will be built for each shop sharing the same X
X_RF = {}
X_train_RF  = {}
X_test_RF  = {}
y_RF  = {}
y_train_RF  = {}
y_test_RF  = {}

for store in df_complete.store_nbr.unique(): 
    X_RF[store] = df_complete.loc[df_complete['store_nbr']==store,cols]
    X_RF[store] = X_RF[store].groupby(by='date').first()
    X_train_RF[store] = X_RF[store].loc[:'2017-08-15']
    X_test_RF[store] = X_RF[store].loc['2017-08-16':]
    y_RF[store] = df_complete.loc[df_complete['store_nbr']==store,['sales','store_nbr','family']].reset_index().set_index(['date','store_nbr','family']).unstack('family')
    y_train_RF[store] = np.log1p(y_RF[store].loc[:'2017-08-15'])
    y_test_RF[store] = y_RF[store].loc['2017-08-16':]

In [215]:
model_RF = {}
y_pred_store_RF = {}
y_fore_store_RF = {}
y_pred_RF = pd.DataFrame()
y_fore_RF = pd.DataFrame()
for store in df_complete.store_nbr.unique():
    weights = X_train_RF[store].year_2017.apply(lambda x: np.exp((-0.9)*1) if x == 0 else np.exp((-0.9)*0))
    model_RF[store] = GridSearchCV(estimator=RandomForestRegressor(random_state=50),
                                    param_grid=dict_params,
                                    refit=True,
                                    verbose=1,
                                    n_jobs=-1,
                                    return_train_score=True,
                                    scoring="neg_mean_absolute_error")
    model_RF[store].fit(X_train_RF[store], y_train_RF[store], sample_weight=weights)
    print(f'Ajustado el modelo de la tienda {store}\n Mejores parámetros {model_RF[store].best_params_}\n')
    y_pred_store_RF[store] = pd.DataFrame(model_RF[store].predict(X_train_RF[store]), index= y_train_RF[store].index, columns=y_train_RF[store].columns)
    y_pred_RF = pd.concat([y_pred_RF, y_pred_store_RF[store]])
    y_fore_store_RF[store] = pd.DataFrame(model_RF[store].predict(X_test_RF[store]), index= y_test_RF[store].index, columns=y_test_RF[store].columns)
    y_fore_RF = pd.concat([y_fore_RF, y_fore_store_RF[store]])

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Ajustado el modelo de la tienda 1
 Mejores parámetros {'bootstrap': True, 'criterion': 'mse', 'max_depth': 15, 'max_features': None, 'max_samples': 0.6, 'n_estimators': 200, 'oob_score': True}

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Ajustado el modelo de la tienda 10
 Mejores parámetros {'bootstrap': True, 'criterion': 'mse', 'max_depth': 20, 'max_features': None, 'max_samples': 0.6, 'n_estimators': 300, 'oob_score': True}

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Ajustado el modelo de la tienda 11
 Mejores parámetros {'bootstrap': True, 'criterion': 'mse', 'max_depth': 15, 'max_features': None, 'max_samples': 0.6, 'n_estimators': 200, 'oob_score': True}

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Ajustado el modelo de la tienda 12
 Mejores parámetros {'bootstrap': True, 'criterion': 'mse', 'max_depth': 20, 'max_features': None, 'max_samples': 0.6, 'n_estimators': 20

Ajustado el modelo de la tienda 39
 Mejores parámetros {'bootstrap': True, 'criterion': 'mse', 'max_depth': 10, 'max_features': None, 'max_samples': 0.6, 'n_estimators': 300, 'oob_score': True}

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Ajustado el modelo de la tienda 4
 Mejores parámetros {'bootstrap': True, 'criterion': 'mse', 'max_depth': 20, 'max_features': None, 'max_samples': 0.6, 'n_estimators': 300, 'oob_score': True}

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Ajustado el modelo de la tienda 40
 Mejores parámetros {'bootstrap': True, 'criterion': 'mse', 'max_depth': 15, 'max_features': None, 'max_samples': 0.6, 'n_estimators': 250, 'oob_score': True}

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Ajustado el modelo de la tienda 41
 Mejores parámetros {'bootstrap': True, 'criterion': 'mse', 'max_depth': 15, 'max_features': None, 'max_samples': 0.6, 'n_estimators': 200, 'oob_score': True}

Fitting 5 folds for each of 12 candida

In [216]:
y_pred_RF = y_pred_RF.stack('family').sort_index()
y_fore_RF = y_fore_RF.stack('family').sort_index()
y_pred_RF = np.exp(y_pred_RF)-1
y_fore_RF = np.exp(y_fore_RF)-1

In [217]:
y_fore_RF

sales
date       store_nbr family                                 
2017-08-16 1         AUTOMOTIVE                     3.884910
                     BABY CARE                      0.000000
                     BEAUTY                         3.773954
                     BEVERAGES                   2395.682991
                     BOOKS                          0.348507
...                                                      ...
2017-08-31 54        POULTRY                       56.448208
                     PREPARED FOODS                74.691356
                     PRODUCE                      565.841313
                     SCHOOL AND OFFICE SUPPLIES     0.186799
                     SEAFOOD                        1.543143

[28512 rows x 1 columns]

In [225]:
y_fore_RF[y_fore_RF<0]=0
store_sales_test = store_sales_test.reset_index().set_index(['date','store_nbr','family']).sort_index()
y_submit2 = y_fore_RF.join(store_sales_test.id)
y_submit2.columns = ['sales','id']
y_submit2 = y_submit2.set_index('id')
y_submit2.sort_index(inplace=True)
y_submit2

,sales
id,
3000888,3.884910
3000889,0.000000
3000890,3.773954
3000891,2395.682991
3000892,0.348507
...,...
3029395,387.981088
3029396,107.432520
3029397,1504.848770


In [219]:
y_submit2.to_csv('submission15.csv')

**El RMSLE es de 0.41417**

# Combinando ambos, Ridge y RF

In [227]:
y_predicciones = pd.DataFrame(y_submit*0.3 + y_submit2*0.7, index= y_submit.index, columns= y_submit.columns)
y_predicciones

,sales
id,
3000888,3.880232
3000889,0.000000
3000890,3.733249
3000891,2534.115002
3000892,0.335306
...,...
3029395,391.239155
3029396,114.917774
3029397,1504.728102


In [228]:
y_predicciones.to_csv('submission16.csv')

**El RMSLE es de 0.40637, y es el mejor resultado obtenido**

Ponderando antes de elevar

In [232]:
y_submit_log = np.log1p(y_submit)
y_submit_log2 = np.log1p(y_submit2)
y_predicciones = pd.DataFrame(y_submit_log*0.3 + y_submit_log2*0.7, index= y_submit.index, columns= y_submit.columns)
y_predicciones = np.exp(y_predicciones)-1
y_predicciones

,sales
id,
3000888,3.880227
3000889,0.000000
3000890,3.732837
3000891,2525.688428
3000892,0.335153
...,...
3029395,391.207812
3029396,114.383096
3029397,1504.728091


In [234]:
y_predicciones.to_csv('submission17.csv')

**El RMSLE es de 0.40772**

Ponderando ahora directamente por 0.5

In [7]:
y_submit = pd.read_csv('submission14.csv')
y_submit2 = pd.read_csv('submission15.csv')
y_submit = y_submit.set_index('id').sort_index()
y_submit2 = y_submit2.set_index('id').sort_index()

In [11]:
y_predicciones = pd.DataFrame(y_submit*0.5 + y_submit2*0.5, index= y_submit.index, columns= y_submit.columns)
y_predicciones

,sales
id,
3000888,3.877114
3000889,0.000000
3000890,3.706112
3000891,2626.403010
3000892,0.326506
...,...
3029395,393.411199
3029396,119.907943
3029397,1504.647657


In [12]:
y_predicciones.to_csv('submission18.csv')

**El RMSLE es de 0.41036**

Se ha visto que empeora respecto a la ponderación 70/30, se va a comprobar por último 80/20; esto habría que haberlo hecho con un set de validación para ver cuál es la ponderación que optimiza el RMSLE, pero por falta de tiempo se están para reentrenar se están enviando directamente los resultados

In [16]:
y_predicciones = pd.DataFrame(y_submit*0.2 + y_submit2*0.8, index= y_submit.index, columns= y_submit.columns)
y_predicciones

,sales
id,
3000888,3.881791
3000889,0.000000
3000890,3.746817
3000891,2487.970998
3000892,0.339707
...,...
3029395,390.153133
3029396,112.422689
3029397,1504.768325


In [17]:
y_predicciones.to_csv('submission19.csv')

**El RMSLE es de 0.40678, ligeramente peor que el 0.40637 que se tenía con el 70/30, por lo que la opción que queda es que entre 50/50 y 80/20 haya una combinación óptima que sea el mínimo absoluto y mejore el 70/30 que se tiene ahora como mejor resultado.**